In [1]:
!cat /proc/meminfo

MemTotal:       32949048 kB
MemFree:        18454848 kB
MemAvailable:   19383220 kB
Buffers:          127716 kB
Cached:          1123200 kB
SwapCached:            0 kB
Active:         13562348 kB
Inactive:         718604 kB
Active(anon):   13030036 kB
Inactive(anon):       48 kB
Active(file):     532312 kB
Inactive(file):   718556 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:                40 kB
Writeback:             0 kB
AnonPages:      13030264 kB
Mapped:           152524 kB
Shmem:                60 kB
Slab:             105396 kB
SReclaimable:      81828 kB
SUnreclaim:        23568 kB
KernelStack:        4304 kB
PageTables:        32264 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    16474524 kB
Committed_AS:   14872224 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

In [2]:
!pip install xgboost

    100% |████████████████████████████████| 15.8MB 3.2MB/s eta 0:00:01
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import gc

from sklearn import preprocessing as prp
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import (accuracy_score, r2_score, confusion_matrix,
f1_score, recall_score, precision_score)

import os
import boto3
import re

import xgboost as xgb

# ruta de trabajo en s3
bucket = 'boi-banregio'

target = 'BMI'
strategy = 'median'
seed = 0

parts = 10
# s3 resource
s3_bucket_resource = boto3.resource('s3').Bucket(bucket)

In [22]:
# Aquí colocamos la ruta de nuestro directorio, el nombre de nuestro archivo y si tiene o no cabecera
prefix = 'datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION'

In [23]:
target_file_name = "TARGETS.csv"
target_file_key = "{}/{}".format(prefix, target_file_name)
target_obj = s3_bucket_resource.Object(target_file_key).get()
data_target = pd.read_csv(io.BytesIO(target_obj['Body'].read())).loc[:,[target]]

In [6]:
# np.random.seed(seed)
# rand_split = np.random.rand(len(data_target))
# train_list = rand_split < 0.6
# val_list = (rand_split >= 0.6) & (rand_split < 0.8)
# test_list = rand_split >= 0.8

# _data_target = {}
# _data_target['train'] = data_target[train_list]
# _data_target['validation'] = data_target[val_list]
# _data_target['test'] = data_target[test_list]

In [26]:
imputer = prp.Imputer(missing_values='NaN', strategy=strategy, axis=0)
scaler = prp.RobustScaler()
for k in range(1, parts):
    data_file_name = "DATA_PART{}.csv".format(k)
    data_file_key = "{}/{}".format(prefix, data_file_name)
    print(data_file_key)
    
    print('Loading data')
    data_part_obj = s3_bucket_resource.Object(data_file_key).get()
    data_part = pd.read_csv(io.BytesIO(data_part_obj['Body'].read()))
    display(data_part.describe())
    #break
    
    print('Impute and scale data')
    imputer = imputer.fit(data_part)
    data_part.loc[:,:] = imputer.transform(data_part)
    data_part.loc[:,:] = scaler.fit_transform(data_part)
    display(data_part.describe())
    
    print('Uploading processed data')
    data_processed_part_file_name = "PROCESSED_DATA/DATA_PREPROCESSED_PART{}.csv".format(k)
    data_processed_part_obj_key =  "{}/{}".format(prefix, data_processed_part_file_name)    

    f_str = io.StringIO()
    data_part.to_csv(f_str, index=False)
    s3_bucket_resource.Object(data_processed_part_obj_key).put(Body=f_str.getvalue())        
    print('Upload step: ', k)
    
    # _data_part = {}
    # _data_part['train'] = data_part[train_list]
    # _data_part['validation'] = data_part[val_list]
    # _data_part['test'] = data_part[test_list]
    
    # print('Uploading processed data')
    # for data_type in ['train', 'validation', 'test']:
    #     data_processed_part_file_name = "PROCESSED_DATA/{}/DATA_PREPROCESSED_PART{}.csv".format(data_type, k)
    #     data_processed_part_obj_key =  "{}/{}".format(prefix, data_processed_part_file_name)
    #     data_procesed_part = pd.concat([_data_target[data_type], _data_part[data_type]], axis=1)
    #     display(data_procesed_part.describe())

    #     f_str = io.StringIO()
    #     data_procesed_part.to_csv(f_str, index=False)
    #     s3_bucket_resource.Object(data_processed_part_obj_key).put(Body=f_str.getvalue())        
    #     print('Upload step: ', k, data_type)
    
    

datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART1.csv
Loading data


,MIN_CRED_LIM_06M,AVG_PND_MAS_CRED_LIM_06M,AVG_PND_MAS_D_CRED_LIM_TO_BAL_12M,MAX_R_BAL_BY_CRED_LIM_03M,AVG_PND_MENOS_R_BAL_BY_CRED_LIM_03M,AVG_PND_MENOS_R_BAL_BY_CRED_LIM_06M,AVG_PND_MAS_R_BAL_BY_CRED_LIM_06M,MAX_R_BAL_BY_CRED_LIM_12M,MIN_R_BAL_BY_CRED_LIM_12M,AVG_R_BAL_BY_MAX_BAL_06M,...,PCT_BK_R_CL_ACC,PCT_BRG_LSE_LSG_CL_ACC,PCT_BRG_LSE_R_CL_ACC,PCT_BRG_LSG_CL_ACC,PCT_BRG_PQ_OP_ACC,PCT_GBN_CS_CL_ACC,PCT_LSE_PQ_CL_ACC,PCT_UN_CRE_OP_ACC,PCT_UN_CRE_OTHER_OP_ACC,PCT_UN_CRE_R_OP_ACC
count,4.252470e+05,4.252470e+05,4.252610e+05,425248.000000,425248.000000,425254.000000,425254.000000,4.252610e+05,425261.000000,425287.000000,...,424082.000000,424082.000000,424082.000000,424082.000000,425287.000000,424082.000000,424082.000000,425287.000000,425287.0,425287.000000
mean,2.092603e+06,2.221805e+06,7.299114e+05,0.854387,0.669450,0.676664,0.690881,1.082443e+00,0.463007,0.832746,...,0.025582,0.000507,0.000008,0.000948,0.005163,0.000002,0.000245,0.001979,0.0,0.000143
std,2.815890e+06,2.938545e+06,1.422928e+06,61.066596,6.559855,5.492461,15.928023,6.427299e+01,0.333100,0.191655,...,0.043502,0.006737,0.000917,0.010088,0.031429,0.000611,0.004150,0.024207,0.0,0.004784
min,2.000000e-02,1.000000e+04,-6.965683e+06,0.000000,0.000000,0.000000,0.000000,3.215434e-08,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,3.249991e+05,3.500000e+05,5.572268e+04,0.554561,0.447826,0.458011,0.456198,7.255577e-01,0.141379,0.761882,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,1.000000e+06,1.050000e+06,2.326161e+05,0.803812,0.709952,0.711587,0.706533,9.110922e-01,0.471844,0.899312,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,2.700000e+06,3.000000e+06,7.985068e+05,0.954410,0.898198,0.894738,0.887690,9.918844e-01,0.749858,0.977211,...,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,1.136910e+08,1.136910e+08,9.866466e+07,38081.000000,3809.009478,2336.058575,9521.006837,3.808100e+04,23.470417,1.000000,...,0.333333,0.250000,0.166667,0.333333,0.500000,0.200000,0.200000,0.919355,0.0,0.357143


Impute and scale data


,MIN_CRED_LIM_06M,AVG_PND_MAS_CRED_LIM_06M,AVG_PND_MAS_D_CRED_LIM_TO_BAL_12M,MAX_R_BAL_BY_CRED_LIM_03M,AVG_PND_MENOS_R_BAL_BY_CRED_LIM_03M,AVG_PND_MENOS_R_BAL_BY_CRED_LIM_06M,AVG_PND_MAS_R_BAL_BY_CRED_LIM_06M,MAX_R_BAL_BY_CRED_LIM_12M,MIN_R_BAL_BY_CRED_LIM_12M,AVG_R_BAL_BY_MAX_BAL_06M,...,PCT_BK_R_CL_ACC,PCT_BRG_LSE_LSG_CL_ACC,PCT_BRG_LSE_R_CL_ACC,PCT_BRG_LSG_CL_ACC,PCT_BRG_PQ_OP_ACC,PCT_GBN_CS_CL_ACC,PCT_LSE_PQ_CL_ACC,PCT_UN_CRE_OP_ACC,PCT_UN_CRE_OTHER_OP_ACC,PCT_UN_CRE_R_OP_ACC
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,...,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.0,425287.000000
mean,0.460000,0.442149,0.669536,0.126485,-0.089929,-0.079965,-0.036273,0.643443,-0.014521,-0.309138,...,0.535701,0.000506,0.000008,0.000945,0.005163,0.000002,0.000245,0.001979,0.0,0.000143
std,1.185590,1.108841,1.915831,152.730825,14.565965,12.576708,36.914918,241.359581,0.547416,0.890055,...,0.912698,0.006727,0.000916,0.010074,0.031429,0.000610,0.004145,0.024207,0.0,0.004784
min,-0.421053,-0.392453,-9.692052,-2.010468,-1.576499,-1.629465,-1.637530,-3.421461,-0.775450,-4.176456,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,-0.284211,-0.264151,-0.238167,-0.623339,-0.582010,-0.580620,-0.580155,-0.696612,-0.543102,-0.638234,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
75%,0.715789,0.735849,0.761833,0.376661,0.417990,0.419380,0.419845,0.303388,0.456898,0.361766,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
max,47.448833,42.506038,132.532500,95244.975553,8456.605300,5347.715607,22065.183644,143003.696770,37.796962,0.467600,...,7.000000,0.250000,0.166667,0.333333,0.500000,0.200000,0.200000,0.919355,0.0,0.357143


Uploading processed data
Upload step:  1
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART2.csv
Loading data


...

Uploading processed data
Upload step:  8
datalake/data/InteligenciaRiesgos/M&M/MCV/DATASETS_3/JAT_MCV_VAR_VARIABLES_SELECCION/PARTITION/DATA_PART9.csv
Loading data


,MIN_BAL_06M,MIN_BAL_12M,AVG_PND_MENOS_CRED_LIM_03M,AVG_CRED_LIM_06M,AVG_D_CRED_LIM_TO_BAL_03M,R_BAL_BY_CRED_LIM,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_03M,AVG_R_BAL_BY_MAX_BAL_03M,BK12_NUM_EXP_PAIDONTIME,NBK12_COMM_PCT_PLUS,...,NUM_LSE_CF_CL_ACC,NUM_LSE_CS_OP_ACC,NUM_LSE_LSG_OP_ACC,NUM_OTHER_FIN_R_CL_ACC,NUM_UN_CRE_LSG_CL_ACC,NUM_UN_CRE_R_OP_ACC,PCT_BK_OP_ACC,PCT_BRG_LSE_OTHER_CL_ACC,PCT_BRG_OTHER_CL_ACC,PCT_LSE_CF_CL_ACC
count,4.252870e+05,4.252870e+05,4.252460e+05,4.252470e+05,4.252480e+05,425234.000000,425287.000000,425287.000000,1.848990e+05,425287.000000,...,424082.0,425287.000000,425287.000000,424082.000000,424082.000000,425287.000000,425287.000000,424082.000000,424082.000000,424082.0
mean,1.152317e+06,9.779195e+05,2.221528e+06,2.219519e+06,7.517257e+05,0.765913,0.880927,0.878536,6.332780e+02,3.895163,...,0.0,0.065572,0.687209,0.098604,0.005433,0.002173,0.152431,0.000002,0.000041,0.0
std,1.793680e+06,1.613709e+06,2.948527e+06,2.935254e+06,1.496209e+06,60.937310,0.168749,0.164714,5.981036e+04,3.017058,...,0.0,1.818258,4.981287,0.311156,0.073892,0.082918,0.178608,0.000457,0.002278,0.0
min,0.000000e+00,0.000000e+00,1.000000e+04,1.000000e+04,-1.005633e+07,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
25%,8.627984e+04,4.796962e+04,3.500000e+05,3.500000e+05,4.769334e+04,0.431561,0.855447,0.845695,1.000000e+00,2.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
50%,4.267568e+05,3.266583e+05,1.050000e+06,1.050000e+06,2.221409e+05,0.715453,0.945045,0.941769,1.000000e+00,3.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.0
75%,1.430796e+06,1.135853e+06,3.000000e+06,3.000000e+06,8.078626e+05,0.911210,0.990703,0.989012,1.000000e+00,6.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0
max,2.001619e+07,1.510410e+07,1.136910e+08,1.136910e+08,9.866466e+07,38081.000000,1.000000,1.000000,1.245483e+07,200.000000,...,0.0,183.000000,502.000000,2.000000,2.000000,9.000000,0.992051,0.111111,0.250000,0.0


Impute and scale data


,MIN_BAL_06M,MIN_BAL_12M,AVG_PND_MENOS_CRED_LIM_03M,AVG_CRED_LIM_06M,AVG_D_CRED_LIM_TO_BAL_03M,R_BAL_BY_CRED_LIM,AVG_PND_MENOS_R_BAL_BY_MAX_BAL_03M,AVG_R_BAL_BY_MAX_BAL_03M,BK12_NUM_EXP_PAIDONTIME,NBK12_COMM_PCT_PLUS,...,NUM_LSE_CF_CL_ACC,NUM_LSE_CS_OP_ACC,NUM_LSE_LSG_OP_ACC,NUM_OTHER_FIN_R_CL_ACC,NUM_UN_CRE_LSG_CL_ACC,NUM_UN_CRE_R_OP_ACC,PCT_BK_OP_ACC,PCT_BRG_LSE_OTHER_CL_ACC,PCT_BRG_OTHER_CL_ACC,PCT_LSE_CF_CL_ACC
count,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,4.252870e+05,425287.000000,...,425287.0,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.000000,425287.0
mean,0.539644,0.598650,0.442043,0.441287,0.696705,0.105206,-0.474050,-0.441210,2.748910e+02,0.223791,...,0.0,0.065572,0.687209,0.098324,0.005418,0.002173,0.165278,0.000002,0.000040,0.0
std,1.334071,1.483348,1.112606,1.107599,1.968467,127.058186,1.247633,1.149296,3.943807e+04,0.754264,...,0.0,1.818258,4.981287,0.310759,0.073788,0.082918,0.714434,0.000456,0.002275,0.0
min,-0.317405,-0.300270,-0.392453,-0.392453,-13.523277,-1.491857,-6.987117,-6.571227,-1.000000e+00,-0.750000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.444444,0.000000,0.000000,0.0
25%,-0.253234,-0.256175,-0.264151,-0.264151,-0.229512,-0.591844,-0.662434,-0.670357,0.000000e+00,-0.250000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,-0.444444,0.000000,0.000000,0.0
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
75%,0.746766,0.743825,0.735849,0.735849,0.770488,0.408156,0.337566,0.329643,0.000000e+00,0.750000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.555556,0.000000,0.000000,0.0
max,14.569874,13.583667,42.506038,42.506038,129.519813,79404.774885,0.406303,0.406312,1.245482e+07,49.250000,...,0.0,183.000000,502.000000,2.000000,2.000000,9.000000,3.523759,0.111111,0.250000,0.0


Uploading processed data
Upload step:  9
